In [1]:


!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-lou_8vq3
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-lou_8vq3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=cc07c2d8dd731d14a3e352a190090a7f69d8f57fc00bd98ba71184692522d4f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgwwnxxr/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:

%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 5
#define BLOCK_SIZE 16

__global__ void matrixMul(int *a, int *b, int *c, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    for (int i = 0; i < width; i++) {
        sum += a[row * width + i] * b[i * width + col];
    }
    c[row * width + col] = sum;
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize matrices
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 2D grid and 2D block
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
    matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print full array
    for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                printf("c[%d][%d] = %d\n", i, j, c[i * N + j]);
            }
        }
    // Print first and last elements of result
    printf("c[0][0] = %d, c[%d][%d] = %d\n", c[0], N-1, N-1, c[(N-1) * N + (N-1)]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

c[0][0] = 0
c[0][1] = 0
c[0][2] = 0
c[0][3] = 0
c[0][4] = 0
c[1][0] = 0
c[1][1] = 0
c[1][2] = 0
c[1][3] = 0
c[1][4] = 0
c[2][0] = 0
c[2][1] = 0
c[2][2] = 0
c[2][3] = 0
c[2][4] = 0
c[3][0] = 0
c[3][1] = 0
c[3][2] = 0
c[3][3] = 0
c[3][4] = 0
c[4][0] = 0
c[4][1] = 0
c[4][2] = 0
c[4][3] = 0
c[4][4] = 0
c[0][0] = 0, c[4][4] = 0

